## TOI ages from vizier

In [2]:
#so just import from another directory
import sys
sys.path.insert(0, '../code')

In [3]:
from kenmon import get_tois_data

df = get_tois_data()
df.tail(3)

,TIC ID,TOI,Previous CTOI,Master,SG1A,SG1B,SG2,SG3,SG4,SG5,...,Stellar Metallicity err,Stellar Mass (M_Sun),Stellar Mass (M_Sun) err,Sectors,Date TOI Alerted (UTC),Date TOI Updated (UTC),Date Modified,Comments,ra_deg,dec_deg
6259,422159302,7183.01,NaN,3,4,3,3,1,4,4,...,NaN,NaN,NaN,"16,17,24,57,76,77,83,84",2025-01-09,2025-01-09,2025-01-16 12:03:02,weak transit shape; possibly SV,335.623083,58.083092
6260,426956677,7184.01,NaN,2,4,2,2,1,4,4,...,0.028,0.990000,0.127349,"57,84",2025-01-09,2025-01-09,2025-01-16 12:02:59,low SNR,2.494583,18.773831
6261,77490011,7185.01,TIC 77490011.01,1,4,1,1,1,4,4,...,NaN,0.202644,0.020103,85,2025-01-09,2025-01-09,2025-01-16 12:03:02,possible odd-even; also a CTOI from Marco Mont...,42.823792,30.281303


In [4]:
df.shape

(6262, 64)

In [5]:
d = df.query("TOI==700.01").squeeze()
d

TIC ID                                 150428135
TOI                                       700.01
Previous CTOI                                NaN
Master                                         2
SG1A                                           5
                                   ...          
Date TOI Updated (UTC)                2024-02-14
Date Modified                2024-02-17 12:12:31
Comments                  TOI-700 c / TOI-700.01
ra_deg                                 97.095708
dec_deg                               -65.578614
Name: 504, Length: 64, dtype: object

In [6]:
d.ra_deg, d.dec_deg

(97.09570833333332, -65.57861388888888)

In [9]:
from kenmon import Target

t = Target(d.ra_deg, d.dec_deg)
p = t.query_vizier_param(param="age")
p

Found 1 references in Vizier using `age`.


{'J/A+A/669/A104/catalog:age': 0.100001}

In [10]:
p.get('J/A+A/669/A104/catalog:age')

0.100001

In [67]:
from kenmon import Target

t = Target(d.ra_deg, d.dec_deg)
# q = r"^(?!.*image).*age.*$"
q = 'age'
t.query_vizier_param(param=q, use_regex=True)

Found 7 references in Vizier using `age`.


{'I/355/paramp:Age-Flame': nan,
 'I/355/paramsup:Age-Flame': nan,
 'J/A+A/662/A15/table1:Age': 1.5,
 'J/A+A/669/A104/catalog:age': 0.100001,
 'J/AJ/167/159/table3:Age': 5.8439,
 'J/AJ/167/159/table3:E_Age': 0.05755,
 'J/AJ/167/159/table3:e_Age': -0.06236}

In [75]:
from tqdm import tqdm

# regex_query = r"^(?!.*image).*age.*$"
regex_query = 'age'

for i,row in tqdm(df.iterrows()):
    t = Target(row.ra_deg, row.dec_deg)
    p = t.query_vizier_param(param=regex_query, use_regex=True)
    data[row.TOI] = p
    break

0it [00:00, ?it/s]WARNING: UnitsWarning: Unit 'x' not supported by the VOUnit standard.  [astropy.units.format.vounit]
0it [00:00, ?it/s]

Found 16 references in Vizier using `age`.


## runtime test

## parallel run

In [76]:
df = get_tois_data()

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor, as_completed
from kenmon import Target
import time
import random

# regex_query = r"^(?!.*image).*age.*$"
regex_query = 'age'

def process_row(row, max_retries=3, timeout=30):
    """Process each row to query Vizier with retries, rate limiting, and timeouts."""
    t = Target(row.ra_deg, row.dec_deg)
    
    for attempt in range(max_retries):
        try:
            # Apply rate limiting with random sleep to prevent server overload
            time.sleep(random.uniform(0.5, 2.0))  
            
            p = t.query_vizier_param(param=regex_query, use_regex=True)
            return row.TOI, p
        except Exception as e:
            print(f"Attempt {attempt+1} failed for TOI {row.TOI}: {e}")
            time.sleep(2 ** attempt)  # Exponential backoff
    return row.TOI, f"Failed after {max_retries} retries"

# Parallel execution with limited workers
data = {}
max_workers = min(4, len(df))  # Adjust based on server limits

with ProcessPoolExecutor(max_workers=max_workers) as executor:
    futures = {executor.submit(process_row, row): row for _, row in df.iterrows()}
    
    for future in tqdm(as_completed(futures), total=len(futures)):
        toi, result = future.result()
        data[toi] = result

In [78]:
import pandas as pd

data_df = pd.DataFrame(data)
data_df.index.unique()

Index(['I/355/paramp:Age-Flame', 'I/355/paramsup:Age-Flame',
       'I/360/goldf:Age-F', 'I/360/goldf:Age-FS', 'J/ApJ/919/138/table1:Age',
       'J/ApJ/919/138/table1:E_Age', 'J/ApJ/919/138/table1:e_Age',
       'J/A+A/669/A104/catalog:age', 'J/A+A/673/A155/galahdr3:Age50',
       'J/A+A/674/A39/table8:AgeStar',
       ...
       'J/A+A/604/A108/rave_dr5:logAgeMean',
       'J/A+A/604/A108/rave_dr5:e_logAgeMean', 'J/A+A/693/L4/tablea1:stAge',
       'J/AJ/167/159/table2:Age', 'J/AJ/167/159/table2:E_Age',
       'J/AJ/167/159/table2:e_Age', 'J/MNRAS/533/1290/table1:Age',
       'J/A+A/692/A243/mrs:Agel', 'J/A+A/692/A243/mrs:Agec',
       'J/A+A/692/A243/lrs:Age'],
      dtype='object', length=389)

## all

In [105]:
data_df.T.dropna(how='all', axis=0).sort_values(by='I/355/paramp:Age-Flame', ascending=True)

,I/355/paramp:Age-Flame,I/355/paramsup:Age-Flame,I/360/goldf:Age-F,I/360/goldf:Age-FS,J/ApJ/919/138/table1:Age,J/ApJ/919/138/table1:E_Age,J/ApJ/919/138/table1:e_Age,J/A+A/669/A104/catalog:age,J/A+A/673/A155/galahdr3:Age50,J/A+A/674/A39/table8:AgeStar,...,J/A+A/604/A108/rave_dr5:logAgeMean,J/A+A/604/A108/rave_dr5:e_logAgeMean,J/A+A/693/L4/tablea1:stAge,J/AJ/167/159/table2:Age,J/AJ/167/159/table2:E_Age,J/AJ/167/159/table2:e_Age,J/MNRAS/533/1290/table1:Age,J/A+A/692/A243/mrs:Agel,J/A+A/692/A243/mrs:Agec,J/A+A/692/A243/lrs:Age
1208.01,0.2,0.930752,0.200482,NaN,NaN,NaN,NaN,1.100806,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1208.02,0.2,0.930752,0.200482,NaN,NaN,NaN,NaN,1.100806,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1208.03,0.2,0.930752,0.200482,NaN,NaN,NaN,NaN,1.100806,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1434.01,0.201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2783.01,0.202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7151.01,NaN,4.1098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7148.01,NaN,9.576713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7158.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7160.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Most ages are in Gyr, some in Myr, some in log10.

In [79]:
data_df.to_csv('../data/TOI_age_vizier.csv')

## load data

In [14]:
import pandas as pd

data_df = pd.read_csv('../data/TOI_age_vizier.csv', index_col=0)
data_df.head()

/tmp/ipykernel_2345253/3736195865.py:3: DtypeWarning: Columns (4,68,69,70,71,72,73,74,75,82,88,91,94,99,131,171,173,174,176,263,267,268,269,275,282,292,293,303,308,309,310,344,386,391,402,404,445,454,462,463,467,483,579,593,620,734,812,836,838,839,875,889,938,943,1002,1026,1069,1071,1072,1074,1112,1119,1166,1168,1170,1201,1203,1204,1206,1251,1278,1294,1307,1329,1346,1386,1467,1471,1531,1532,1559,1562,1564,1568,1645,1818,3576,3647,3658,3693,3695,3698,3787,4241,4248,4249,4289,4298,4432,4761,4801,4851,4916,4917,4919,4921,4926,4945,4947,5065,5079,5098,5116,5132,5359,5638,5784,5785,5789,5803,5924,5984,6045,6046,6061) have mixed types. Specify dtype option on import or set low_memory=False.
  data_df = pd.read_csv('../data/TOI_age_vizier.csv', index_col=0)


,101.01,103.01,104.01,102.01,105.01,107.01,106.01,109.01,110.01,108.01,...,7176.01,7177.01,7178.01,7179.01,7182.01,7181.01,7180.01,7183.01,7184.01,7185.01
I/355/paramp:Age-Flame,5.25700,4.343000,NaN,3.674,10.938000,4.136000,1.829,NaN,7.222000,13.261,...,NaN,7.489000,5.736,5.842000,4.803000,3.836000,7.770000,NaN,2.597000,NaN
I/355/paramsup:Age-Flame,NaN,2.109727,3.198016,1.9911742,7.112666,3.843425,NaN,NaN,2.565268,NaN,...,NaN,NaN,NaN,5.661383,8.089615,3.873336,3.261123,NaN,0.886076,NaN
I/360/goldf:Age-F,5.25653,NaN,NaN,3.6736262,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,7.489475,NaN,NaN,4.803039,NaN,7.769972,NaN,2.596616,NaN
I/360/goldf:Age-FS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
J/ApJ/919/138/table1:Age,4.89000,NaN,NaN,NaN,5.620000,NaN,NaN,NaN,2.940000,1.590,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
refs = {}
for i in data_df.index:
    count=data_df.loc[i].dropna().shape
    refs[i] = count

In [16]:
refs = pd.Series(refs).sort_values(ascending=False)
refs.head(20)

I/355/paramp:Age-Flame               (3944,)
J/A+A/669/A104/catalog:age           (2528,)
I/355/paramsup:Age-Flame             (2212,)
I/360/goldf:Age-F                     (857,)
J/A+A/673/A155/lamostl7:Age50         (302,)
J/other/Nat/586.528/table1:Tage       (287,)
J/A+A/671/A16/table9:plate-image      (187,)
J/A+A/671/A16/table8:plate-image      (186,)
J/MNRAS/445/4395/tablea1:Age          (151,)
J/other/Nat/586.528/table1:e_Tage     (137,)
J/ApJ/835/61/table4:LogAge            (122,)
J/A+A/673/A155/ravedr6:Age50          (117,)
J/A+A/673/A155/apogee17:Age50         (110,)
VII/293/catalog:log10Age              (110,)
J/ApJ/919/138/table1:E_Age            (104,)
J/ApJ/919/138/table1:e_Age            (104,)
J/ApJ/919/138/table1:Age              (104,)
J/AJ/164/26/table1:Age                 (98,)
J/MNRAS/422/2024/tables:Age            (98,)
J/A+A/673/A155/lamostm7:Age50          (98,)
dtype: object

https://cdsarc.cds.unistra.fr/viz-bin/cat/I/355

In [91]:
# Age (Gyr) of the star from FLAME using stellar models
age_flame = data_df.loc['I/355/paramp:Age-Flame']
age_flame.sort_values()

1208.01      0.2
1208.02      0.2
1208.03      0.2
1434.01    0.201
2783.01    0.202
           ...  
7166.01      NaN
7173.01      NaN
7176.01      NaN
7183.01      NaN
7185.01      NaN
Name: I/355/paramp:Age-Flame, Length: 6262, dtype: object

https://cdsarc.cds.unistra.fr/viz-bin/cat/J/A+A/669/A104

In [95]:
age_gaia = data_df.loc['J/A+A/669/A104/catalog:age']
age_gaia.sort_values()

1693.01         0.1
700.04     0.100001
700.03     0.100001
700.02     0.100001
700.01     0.100001
             ...   
7181.01         NaN
7180.01         NaN
7183.01         NaN
7184.01         NaN
7185.01         NaN
Name: J/A+A/669/A104/catalog:age, Length: 6262, dtype: object

https://cdsarc.cds.unistra.fr/viz-bin/cat/J/A+A/673/A155

In [97]:
# there are Age16 and Age84 in table but not shown by default
age_lamost7 = data_df.loc['J/A+A/673/A155/lamostl7:Age50']
age_lamost7.sort_values()

5439.01    0.425001
5270.01    0.726045
5066.01     0.82253
7088.01    0.824189
6395.01    0.826276
             ...   
7181.01         NaN
7180.01         NaN
7183.01         NaN
7184.01         NaN
7185.01         NaN
Name: J/A+A/673/A155/lamostl7:Age50, Length: 6262, dtype: object

https://cdsarc.cds.unistra.fr/viz-bin/cat/J/ApJ/898/27

In [107]:
data_df.loc['J/ApJ/898/27/table2:_50Age2'].dropna()

1860.01    338.0
Name: J/ApJ/898/27/table2:_50Age2, dtype: object

In [108]:
# using rhk; in Myr
cols = ['J/ApJ/898/27/table2:_16Age1','J/ApJ/898/27/table2:_50Age1','J/ApJ/898/27/table2:_84Age1']
# convert 
age_baffles_rhk = data_df.loc[cols].dropna(axis=1).T.apply(lambda x: x/1e3)
age_baffles_rhk.sort_values(by='J/ApJ/898/27/table2:_50Age1')

,J/ApJ/898/27/table2:_16Age1,J/ApJ/898/27/table2:_50Age1,J/ApJ/898/27/table2:_84Age1
200.01,0.0177,0.0888,0.625
1860.01,0.103,0.285,1.25
1726.01,0.199,0.49,1.77
1726.02,0.199,0.49,1.77
5082.01,0.273,0.641,2.12
179.01,0.293,0.681,2.21
2666.01,0.382,0.858,2.6
6902.01,0.449,0.986,2.86
5140.01,0.991,1.98,4.63
4537.01,1.1,2.17,4.92


TOI-1726: https://ui.adsabs.harvard.edu/abs/2023A%26A...672A.126D/abstract

## reliable

In [34]:
cols = ['I/355/paramp:Age-Flame',
        'J/A+A/669/A104/catalog:age',
        'J/A+A/673/A155/lamostl7:Age50',
        'J/A+A/673/A155/ravedr6:Age50', 
        'J/A+A/673/A155/apogee17:Age50',
        'J/ApJ/898/27/table2:_50Age1'
       ]
# drop rows with NaN
data_df2 = data_df.loc[cols].T.astype(float)
data_df2['J/ApJ/898/27/table2:_50Age1'] = data_df2['J/ApJ/898/27/table2:_50Age1'].apply(lambda x: x/1e3)
data_df2 = data_df2.dropna(how='all', axis=0)
data_df2

,I/355/paramp:Age-Flame,J/A+A/669/A104/catalog:age,J/A+A/673/A155/lamostl7:Age50,J/A+A/673/A155/ravedr6:Age50,J/A+A/673/A155/apogee17:Age50,J/ApJ/898/27/table2:_50Age1
101.01,5.257,NaN,NaN,NaN,NaN,NaN
103.01,4.343,2.469392,NaN,NaN,NaN,NaN
104.01,NaN,3.349768,NaN,3.211976,NaN,NaN
102.01,3.674,3.248167,NaN,NaN,NaN,NaN
105.01,10.938,5.051802,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
7179.01,5.842,NaN,NaN,NaN,NaN,NaN
7182.01,4.803,NaN,NaN,NaN,NaN,NaN
7181.01,3.836,NaN,NaN,NaN,NaN,NaN
7180.01,7.770,NaN,NaN,NaN,NaN,NaN


In [44]:
import numpy as np

data_df_median = data_df2.apply(lambda x: np.nanmedian(x), axis=1)
data_df_median.dropna().shape

(4673,)

In [45]:
data_df2_mean = data_df2.mean(axis=1)
data_df2_mean.dropna().shape

(4673,)

In [152]:
short_list = data_df2_mean[data_df2_mean<1].sort_values()
short_list = set([int(str(i).split('.')[0]) for i in list(short_list.index)])
short_list

{137,
 159,
 199,
 231,
 233,
 277,
 314,
 386,
 451,
 503,
 508,
 522,
 540,
 579,
 602,
 616,
 621,
 624,
 626,
 629,
 642,
 671,
 692,
 700,
 703,
 761,
 769,
 772,
 779,
 782,
 815,
 863,
 885,
 905,
 952,
 986,
 989,
 990,
 991,
 1027,
 1044,
 1059,
 1105,
 1125,
 1139,
 1150,
 1151,
 1196,
 1198,
 1208,
 1218,
 1224,
 1239,
 1311,
 1322,
 1327,
 1337,
 1355,
 1364,
 1382,
 1409,
 1418,
 1434,
 1468,
 1487,
 1489,
 1518,
 1531,
 1533,
 1564,
 1599,
 1641,
 1687,
 1693,
 1801,
 1803,
 1853,
 1901,
 1916,
 1924,
 1926,
 1927,
 1976,
 2051,
 2063,
 2064,
 2080,
 2085,
 2095,
 2098,
 2123,
 2210,
 2241,
 2279,
 2283,
 2327,
 2394,
 2431,
 2434,
 2435,
 2448,
 2457,
 2481,
 2484,
 2495,
 2504,
 2541,
 2551,
 2552,
 2577,
 2592,
 2594,
 2618,
 2666,
 2672,
 2692,
 2757,
 2783,
 2809,
 2813,
 2831,
 2832,
 2841,
 2847,
 2854,
 2869,
 2872,
 2898,
 2901,
 2914,
 2945,
 2953,
 2954,
 2961,
 2979,
 3009,
 3029,
 3045,
 3047,
 3051,
 3067,
 3068,
 3076,
 3091,
 3104,
 3139,
 3143,
 3152,
 31